In [260]:
# Import dependencies
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [261]:
# Load the data
charity_df = pd.read_csv("Resources/charity_data.csv")
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [262]:
# Drop the EIN and NAME columns
charity_df = charity_df.drop(["EIN", "NAME"], axis=1)
charity_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [263]:
# Generate the categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

In [264]:
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [265]:
### Bucketing ###

In [266]:
# Bucket the APPLICATION_TYPE column
application_counts = charity_df["APPLICATION_TYPE"].value_counts()

replace_applications = list(
    application_counts[application_counts < 100].index)

for applications in replace_applications:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(applications, "Other")

In [267]:
# Bucket the CLASSIFICATION column
classification_counts = charity_df["CLASSIFICATION"].value_counts()

replace_classifications = list(
    classification_counts[classification_counts < 500].index)

for classifications in replace_classifications:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classifications, "Other")

In [268]:
### Encoding ###

In [269]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [270]:
# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(
    enc.fit_transform(charity_df[charity_cat]))

In [271]:
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [272]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df, left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat, 1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [273]:
### Splitting/Standardizing ###

In [274]:
# Split the preprocessed data into features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop("IS_SUCCESSFUL", 1).values

In [275]:
# Split the preprocessed data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)

In [276]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [277]:
### Building the model ###

In [278]:
# Define the model - deep neural network
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"
    )
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer2, activation="relu"
    )
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 8)                 368       
_________________________________________________________________
dense_15 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 6         
Total params: 419
Trainable params: 419
Non-trainable params: 0
_________________________________________________________________


In [279]:
### Compile, train and evaluate the model ###

In [280]:
# Compile the model
nn.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [281]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 15461.8438 - accuracy: 0.5031
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 10052.3467 - accuracy: 0.4996
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 6102.5879 - accuracy: 0.5189
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 6525.9619 - accuracy: 0.5148
Epoch 5/100
804/804 [==============================] - ETA: 0s - loss: 9638.4424 - accuracy: 0.512 - 2s 3ms/step - loss: 9568.7012 - accuracy: 0.5124
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 12367.9512 - accuracy: 0.4996
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 4806.0972 - accuracy: 0.5019
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 6392.3633 - accuracy: 0.5330
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 5200.5220 - accuracy: 0.5154
Epoch 10/100
80

804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 81/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6911 - accuracy: 0.5318
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 

In [282]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6908 - accuracy: 0.5343
Loss: 0.6907925605773926, Accuracy: 0.5343440175056458


In [283]:
### Rebuild the model with more neurons ###

In [284]:
# Define the model - deep neural network
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 35

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"
    )
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer2, activation="relu"
    )
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 100)               4600      
_________________________________________________________________
dense_18 (Dense)             (None, 35)                3535      
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 36        
Total params: 8,171
Trainable params: 8,171
Non-trainable params: 0
_________________________________________________________________


In [285]:
# Compile the model
nn.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [287]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5318
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5318
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 

In [288]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6908 - accuracy: 0.5343
Loss: 0.6908087730407715, Accuracy: 0.5343440175056458


In [289]:
# Rebuild the model with more hidden layers ###

In [290]:
# Define the model - deep neural network
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"
    )
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer2, activation="relu"
    )
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer3, activation="relu"
    )
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 20)                920       
_________________________________________________________________
dense_21 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_22 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 6         
Total params: 1,191
Trainable params: 1,191
Non-trainable params: 0
_________________________________________________________________


In [291]:
# Compile the model
nn.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [292]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 20117.5352 - accuracy: 0.4795
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 1518.2853 - accuracy: 0.4997
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 2514.1729 - accuracy: 0.5045
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 8507.4814 - accuracy: 0.4950
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 2179.5947 - accuracy: 0.5058
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 486.5527 - accuracy: 0.5113
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.7134 - accuracy: 0.5318
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6960 - accuracy: 0.5318
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6917 - accuracy: 0.5318
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - los

804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 81/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 82/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 83/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5318
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 

In [293]:
### Rebuild the model with a different output activation function ###

In [295]:
# Define the model - deep neural network
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"
    )
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(
        units=hidden_nodes_layer2, activation="relu"
    )
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 8)                 368       
_________________________________________________________________
dense_28 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 6         
Total params: 419
Trainable params: 419
Non-trainable params: 0
_________________________________________________________________


In [296]:
# Compile the model
nn.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [297]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - ETA: 0s - loss: 8.2024 - accuracy: 0.46 - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 10/100
804/804 [===========================

804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 81/100
804/804 [==============================] - 2s 2ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 8.2024 - accuracy: 